In [3]:
import coco_text
import numpy as np
import cv2
import string
import json
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

## Prepare training set for CRNN

In [2]:
dataDir='../train2014'
dataType='train2014'
outputDir='../CRNN_COCO'
char_dict = string.digits+string.ascii_letters+string.punctuation

In [3]:
ct = coco_text.COCO_Text('COCO_Text.json')

imgs = ct.getImgIds(imgIds=ct.train, 
                    catIds=[('legibility','legible'),('class','machine printed'),
                           ('language','english')])

ann_json = {}
ann_json['abc'] = char_dict

loading annotations into memory...
0:00:02.838658
creating index...
index created!


In [4]:
NUM_IMAGES = len(imgs)
NUM_TRAINIMGS = 10000
NUM_TESTIMGS = 3000
imgs_info = ct.loadImgs(imgs)
count_train = 0
count_test = 0

# write train images
ann_json['train'] = [] 
for i in range(NUM_TRAINIMGS):
    I = cv2.imread('%s/%s'%(dataDir,imgs_info[i]['file_name']))
    annIds = ct.getAnnIds(imgIds=imgs_info[i]['id'],catIds=[('legibility','legible'),('class','machine printed'),
                           ('language','english')],areaRng=[300,10000])
    if len(annIds)>0:
        ann = ct.loadAnns(annIds)
    for j in range(len(ann)):
        bbox = ann[j]['bbox']
        bbox = [int(ele) for ele in bbox]
        if bbox[1]+bbox[3] >= I.shape[0] or bbox[0]+bbox[2] >= I.shape[1] or bbox[2]<=0 or bbox[3]<=0:
            continue
        cv2.imwrite('%s/train%s_%s'%(outputDir,str(j),imgs_info[i]['file_name']),
                    I[bbox[1]:(bbox[1]+bbox[3]),bbox[0]:(bbox[0]+bbox[2]),:])
        ann_json['train'].append({'text':ann[j]['utf8_string'],
                                'name':'train%s_%s'%(str(j),imgs_info[i]['file_name'])})
        count_train += 1

## write test images
ann_json['test'] = [] 
for i in range(NUM_TRAINIMGS,NUM_TRAINIMGS+NUM_TESTIMGS,1):
    I = cv2.imread('%s/%s'%(dataDir,imgs_info[i]['file_name']))
    annIds = ct.getAnnIds(imgIds=imgs_info[i]['id'],catIds=[('legibility','legible'),('class','machine printed'),
                           ('language','english')],areaRng=[300,10000])
    if len(annIds)>0:
        ann = ct.loadAnns(annIds)
    for j in range(len(ann)):
        bbox = ann[j]['bbox']
        bbox = [int(ele) for ele in bbox]
        if bbox[1]+bbox[3] >= I.shape[0] or bbox[0]+bbox[2] >= I.shape[1] or bbox[2]<=0 or bbox[3]<=0:
            continue
        cv2.imwrite('%s/test%s_%s'%(outputDir,str(j),imgs_info[i]['file_name']),
                    I[bbox[1]:(bbox[1]+bbox[3]),bbox[0]:(bbox[0]+bbox[2]),:])
        ann_json['test'].append({'text':ann[j]['utf8_string'],
                                'name':'test%s_%s'%(str(j),imgs_info[i]['file_name'])})
        count_test += 1

with open(outputDir+'/desc.json', 'w') as outfile:  
    json.dump(ann_json, outfile)

print("number of training instances %d"%count_train)
print("number of testing instances %d"%count_test)

number of training instances 33190
number of testing instances 10179


### For Debug

In [103]:
with open(outputDir+'/desc.json') as infile:  
    myanns = json.load(infile)

In [104]:
for i in range(len(myanns['train'])):
    I = cv2.imread(outputDir+'/'+myanns['train'][i]['name'])
    print(I.shape)

(11, 29, 3)
(36, 22, 3)
(19, 116, 3)
(31, 43, 3)
(24, 51, 3)
(14, 36, 3)
(22, 24, 3)
(22, 79, 3)
(7, 71, 3)
(14, 97, 3)
(13, 56, 3)
(14, 122, 3)
(13, 95, 3)
(65, 37, 3)
(17, 32, 3)
(18, 46, 3)
(39, 50, 3)
(31, 64, 3)
(8, 52, 3)
(10, 32, 3)
(26, 155, 3)
(12, 42, 3)
(13, 30, 3)
(11, 37, 3)
(45, 20, 3)
(16, 24, 3)
(36, 12, 3)
(45, 20, 3)
(16, 24, 3)
(36, 12, 3)
(45, 20, 3)
(16, 24, 3)
(36, 12, 3)
(32, 44, 3)
(32, 32, 3)
(33, 66, 3)
(14, 23, 3)
(23, 36, 3)
(20, 59, 3)
(16, 45, 3)
(13, 33, 3)
(50, 31, 3)
(42, 24, 3)
(31, 19, 3)
(43, 69, 3)
(27, 27, 3)
(19, 35, 3)
(39, 35, 3)
(24, 37, 3)
(104, 88, 3)
(27, 46, 3)
(54, 117, 3)
(22, 29, 3)
(25, 25, 3)
(19, 25, 3)
(19, 32, 3)
(18, 30, 3)
(17, 28, 3)
(19, 38, 3)
(22, 22, 3)
(23, 29, 3)
(22, 18, 3)
(20, 31, 3)
(18, 49, 3)
(43, 60, 3)
(28, 38, 3)
(14, 34, 3)
(46, 135, 3)
(14, 75, 3)
(12, 67, 3)
(10, 32, 3)
(14, 34, 3)
(46, 135, 3)
(14, 75, 3)
(12, 67, 3)
(19, 22, 3)
(18, 53, 3)
(41, 57, 3)
(48, 40, 3)
(28, 30, 3)
(20, 21, 3)
(28, 22, 3)
(28, 27, 3)

## Prepare training set for CTPN

In [12]:
dataDir = '../train2014'
dataType = 'train2014'
outputDir = '../CTPN_COCO'

In [5]:
ct = coco_text.COCO_Text('COCO_Text.json')

imgs = ct.getImgIds(imgIds=ct.train, 
                    catIds=[('legibility','legible'),('class','machine printed'),
                           ('language','english')])


loading annotations into memory...
0:00:06.181307
creating index...
index created!


In [14]:
NUM_IMAGES = len(imgs)
NUM_TRAINIMGS = 10000
NUM_TESTIMGS = 3000
imgs_info = ct.loadImgs(imgs)
count_train = 0
count_test = 0

In [15]:
for i in range(NUM_TRAINIMGS):
    I = cv2.imread('%s/%s'%(dataDir,imgs_info[i]['file_name']))
    annIds = ct.getAnnIds(imgIds=imgs_info[i]['id'],catIds=[('legibility','legible'),('class','machine printed'),
                           ('language','english')],areaRng=[300,10000])
    if len(annIds)>0:
        ann = ct.loadAnns(annIds)
        # write images to ./outputDir/image folder
        cv2.imwrite('%s/image/%s'%(outputDir,imgs_info[i]['file_name']),I)
        # write images to ./outputDir/label folder
        with open('%s/label/gt_%s.txt'%(outputDir,imgs_info[i]['file_name'][0:-4]),'w') as f:
            for j in range(len(ann)):        
                bbox = ann[j]['bbox']
                bbox = [int(ele) for ele in bbox]
                if bbox[1]+bbox[3] >= I.shape[0] or bbox[0]+bbox[2] >= I.shape[1] or bbox[2]<=0 or bbox[3]<=0:
                    continue
                cordns = [bbox[0],bbox[1],bbox[0],bbox[1]+bbox[3],
                          bbox[0]+bbox[2],bbox[1]+bbox[3],bbox[0]+bbox[2],bbox[1]]
                cordns = [str(item) for item in cordns]
                st=','.join(cordns)
                f.write(st+'\n')
        count_train += 1
    if i % 100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
